# DAILY SQL PROGRESS LOG 

To be used with notion. Contains daily sql questions / repetitions.


## January 15, 2026

In [ ]:
-- Histogram of Tweets - Easy 
WITH TWEETER AS(
SELECT 
  count(tweet_id) as tweet_bucket,  
  user_id
FROM tweets
WHERE EXTRACT (YEAR FROM tweet_date) = 2022
GROUP BY user_id
)
SELECT 
  tweet_bucket, 
  COUNT(user_id) AS users_num
FROM TWEETER
GROUP BY tweet_bucket

In [ ]:
-- Fill missing client data
with cte as(
SELECT *, 
  Count(category) OVER (order by product_id) as category_number
  
FROM products)

SELECT product_id, 
  COALESCE(category, MAX(category) OVER (PARTITION BY category_number
                                          ORDER BY product_id)), 
  name
  
FROM cte;

# 19th January, 2026 

In [ ]:
WITH CTE AS (
SELECT
  user_id, 
  row_number()over(partition by user_id order by transaction_date) AS transact,
  transaction_date,
  spend 
FROM transactions
)
SELECT 
  user_id, 
  spend, 
  transaction_date
FROM CTE 
WHERE transact=3

# 20th January. 2026

In [ ]:
SELECT 
  EXTRACT (MONTH FROM event_date) as mth, 
  COUNT( DISTINCT user_id) as monthly_active_users 
FROM user_actions 
WHERE user_id IN (
                  SELECT DISTINCT user_id 
                  FROM user_actions 
                  WHERE EXTRACT(MONTH FROM event_date)=6
                  AND event_type IN ('sign-in','comment','like')
                  )
AND EXTRACT(MONTH FROM event_date) = 7
AND event_type IN ('sign-in','comment','like')
GROUP BY mth

In [ ]:
WITH YEAR_DATA AS(
SELECT 
  EXTRACT (YEAR FROM transaction_date) as yr, 
  product_id, 
  SUM(spend) OVER (
                    PARTITION BY product_id, transaction_date
                  ) AS curr_year_spend, 
  LAG(SUM(spend))OVER (
                        PARTITION BY product_id
                      ) AS prev_year_spend
FROM user_transactions 
GROUP BY  transaction_date, product_id, spend
)
SELECT 
  yr, 
  product_id, 
  curr_year_spend,
  prev_year_spend,
  ROUND(((curr_year_spend - prev_year_spend)::DECIMAL/prev_year_spend)*100,2) AS yoy_rate
  FROM YEAR_DATA

# 21st January, 2026

In [ ]:
--option 1 
WITH summary AS (  
  SELECT  
    item_type,  
    SUM(square_footage) AS total_sqft,  
    COUNT(*) AS item_count  
  FROM inventory  
  GROUP BY item_type
),
prime_occupied_area AS (  
  SELECT  
    item_type,
    total_sqft,
    FLOOR(500000/total_sqft) AS prime_item_batch_count,
    (FLOOR(500000/total_sqft) * item_count) AS prime_item_count
  FROM summary  
  WHERE item_type = 'prime_eligible'
)

SELECT
  item_type,
  CASE 
    WHEN item_type = 'prime_eligible' 
      THEN (FLOOR(500000/total_sqft) * item_count)
    WHEN item_type = 'not_prime' 
      THEN FLOOR((500000 - (SELECT FLOOR(500000/total_sqft) * total_sqft FROM prime_occupied_area)) / total_sqft) * item_count
  END AS item_count
FROM summary
ORDER BY item_type DESC;
--option 2 
WITH summary AS (  
  SELECT  
    item_type,  
    SUM(square_footage) AS total_sqft,  
    COUNT(*) AS item_count  
  FROM inventory  
  GROUP BY item_type
),
prime_occupied_area AS (  
  SELECT  
    item_type,
    total_sqft,
    FLOOR(500000/total_sqft) AS prime_item_batch_count,
    (FLOOR(500000/total_sqft) * item_count) AS prime_item_count
  FROM summary  
  WHERE item_type = 'prime_eligible'
)

SELECT
  item_type,
  CASE 
    WHEN item_type = 'prime_eligible' 
      THEN (FLOOR(500000/total_sqft) * item_count)
    WHEN item_type = 'not_prime' 
      THEN FLOOR((500000 - (SELECT FLOOR(500000/total_sqft) * total_sqft FROM prime_occupied_area)) / total_sqft) * item_count
  END AS item_count
FROM summary
ORDER BY item_type DESC;

In [ ]:
SELECT 
  salary AS second_highest_salary
FROM employee
WHERE salary < (
                SELECT 
                  MAX(salary)
                FROM employee
               )
ORDER BY salary DESC
LIMIT 1

In [ ]:
WITH expansion AS(
    SELECT 
        searches
    FROM search_frequency
    GROUP BY searches,
    GENERATE_SERIES(1,num_users)
)
SELECT 
    ROUND((PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY searches))::DECIMAL,1) as median
FROM expansion

# 22nd January, 2026

In [ ]:
SELECT 
  COALESCE(adv.user_id, dp.user_id) AS user_id,
  CASE 
    WHEN dp.paid is NULL THEN 'CHURN' 
    WHEN dp.paid IS NOT NULL AND adv.status IN ('NEW','EXISTING','RESURRECT') THEN 'EXISTING'
    WHEN dp.paid IS NOT NULL AND adv.status = 'CHURN' THEN 'RESURRECT'
    ELSE 'NEW'
    END AS new_status
FROM advertiser AS adv 
FULL OUTER JOIN daily_pay AS dp 
ON adv.user_id = dp.user_id
ORDER BY user_id ASC

In [ ]:
WITH SUMMARY AS(
SELECT 
  user_id, 
  LAG(EXTRACT(YEAR FROM filing_date)) OVER(
  PARTITION BY user_id ORDER BY EXTRACT(YEAR FROM filing_date))
  AS prev_year,
  EXTRACT(YEAR FROM filing_date) AS mid_year,
  LEAD(EXTRACT(YEAR FROM filing_date)) OVER(
  PARTITION BY user_id ORDER BY EXTRACT(YEAR FROM filing_date))
  AS next_year
FROM filed_taxes
WHERE product LIKE '%TurboTax%'
ORDER BY user_id
) 
SELECT 
  DISTINCT user_id 
FROM SUMMARY 
WHERE prev_year IS NOT NULL AND prev_year = mid_year-1
AND next_year IS NOT NULL AND next_year = mid_year +1

In [ ]:
WITH SUMMARY AS(
SELECT 
  contact_id, 
  event_type, 
  LAG(EXTRACT(WEEK FROM event_date))OVER(
  PARTITION BY contact_id ORDER BY event_date)
  AS prev_week,
  EXTRACT(WEEK FROM event_date) AS mid_week,
  LEAD(EXTRACT(WEEK FROM event_date))OVER(
  PARTITION BY contact_id ORDER BY event_date)
  AS next_week
FROM marketing_touches
ORDER BY contact_id
)
SELECT 
  DISTINCT crm.email
FROM crm_contacts AS crm
JOIN SUMMARY AS s
ON s.contact_id = crm.contact_id
WHERE prev_week IS NOT NULL AND prev_week = mid_week - 1 
AND next_week IS NOT NULL AND next_week = mid_week + 1 
AND s.contact_id IN (
                            SELECT contact_id 
                            FROM marketing_touches 
                            WHERE event_type = 'trial_request'
                          )


In [ ]:
SELECT 
  CONCAT_WS(',',p1.topping_name,p2.topping_name,p3.topping_name ) AS pizza, 
  p1.ingredient_cost+p2.ingredient_cost+p3.ingredient_cost AS total_cost
FROM pizza_toppings AS p1 
JOIN pizza_toppings AS p2
ON p1.topping_name < p2.topping_name 
JOIN pizza_toppings AS p3 
ON p2.topping_name < p3.topping_name
ORDER BY total_cost DESC, pizza ASC

# 24th January 2026 

In [ ]:
with cte as (
SELECT
  item_count
FROM items_per_order 
GROUP BY 
  item_count, 
  GENERATE_SERIES(1,order_occurrences)
ORDER BY item_count
)
select 
  ROUND((PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY item_count))::DECIMAL,1)
from cte
  

In [ ]:
WITH running_orders AS (
SELECT
  *,
  SUM(order_occurrences) OVER (
    ORDER BY item_count ASC) as running_sum,
  SUM(order_occurrences) OVER () AS total_sum
FROM items_per_order
)

SELECT ROUND(AVG(item_count)::DECIMAL,1) AS median
FROM running_orders
WHERE total_sum <= 2 * running_sum
  AND total_sum >= 2 * (running_sum - order_occurrences);

# 25th January, 2026

In [ ]:
WITH COUNTER AS 
(
  SELECT
    candidate_id,
    COUNT(skill) AS skill_count
  FROM candidates
  WHERE skill IN ('Python', 'Tableau', 'PostgreSQL')
  GROUP BY candidate_id
)
SELECT
  candidate_id 
FROM COUNTER 
WHERE skill_count = 3

# 26th January, 2026 

In [ ]:
SELECT 
  p.page_id
FROM pages AS p 
LEFT JOIN page_likes AS pl
ON p.page_id = pl.page_id
WHERE pl.liked_date is NULL
ORDER BY p.page_id ASC